In [2]:
import napari
import PIL.Image
import numpy as np

PIL.Image.MAX_IMAGE_PIXELS = 1056323868

v = napari.Viewer()

# lab = np.zeros((109, 2800, 1987), dtype='int')
# lab[10:90,100:2700,100:1900] = 1
# v.add_labels(lab, scale=(100,3.5,3.5))

In [ ]:
import zarr
from skimage import io
from skimage import measure
from tqdm import tqdm
import h5py
import numpy as np

## convert ilastik output to labels in tif format

In [ ]:


f = h5py.File(
    "/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/191815_05401_CH1_Simple Segmentation.h5",
    "r",
)
pred = f.get("exported_data")
pred = pred[:, :, 0]
mask = pred == 1
mask = mask.astype("int32")
labels = measure.label(mask)
io.imsave("/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/54_comps.tif", labels)

## write zarr

In [ ]:
zarr_labels = zarr.zeros((109, 28001, 19868), chunks=(1,1000,1000), dtype='i4')

for z in tqdm(range(52, 57)):
    labels = io.imread("/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/" + str(z) + "_comps.tif")
    zarr_labels[z,:,:] = labels

zarr.save('/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/labels_zarr.zarr', zarr_labels)

In [ ]:
zarr_image = zarr.zeros((109, 28001, 19868), chunks=(1,1000,1000), dtype='i4')

for z in tqdm(range(52, 57)):
    image = io.imread("/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/191815_0" + str(z) + "01_CH1.tif")
    zarr_image[z,:,:] = image

#zarr.save('/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/image_zarr.zarr', zarr_image)

In [ ]:
import zarr
zarr_image = zarr.open("/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/labels_zarr.zarr")
zarr_image.shape


In [ ]:
zarr_image.shape

In [ ]:
for x1 in range(0, 20000,2000):
    x2 = np.amin([x1+2000, zarr_image.shape[2]])
    im = zarr_image[52:57,:,x1:x2]
    fname = "/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/ilastik/" + str(x1) + ".tif"
    io.imsave(fname, im)

In [ ]:
zarr_probs = zarr.zeros((109, 28001, 19868), chunks=(1,1000,1000), dtype='float')
for x1 in range(0, 20000,2000):
    x2 = np.amin([x1+2000, zarr_image.shape[2]])
    f = h5py.File(
        "/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/ilastik/" + str(x1) + "_Probabilities.h5",
        "r",
    )
    pred = f.get("exported_data")
    probs = pred[:,:,:,0]
    zarr_probs[52:57,:,x1:x2] = probs
    break


zarr.save('/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/probs_zarr.zarr', zarr_probs)

# Viterbrain

In [ ]:
from brainlit.algorithms.generate_fragments.state_generation import state_generation

In [ ]:
sg = state_generation(
    image_path="/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/image_zarr.zarr",
    ilastik_program_path="/Applications/ilastik-1.4.0b21-OSX.app/Contents/ilastik-release/run_ilastik.sh",
    ilastik_project_path="/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/ilastik/bil_slice.ilp",
    chunk_size=[5,1000,1000],
    soma_coords=[],
    parallel=8,
    prob_path = "/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/probs_zarr.zarr",
)

In [ ]:
sg.compute_frags()

## Check labels progress

In [10]:
from skimage import io
import napari
import PIL.Image
import numpy as np


im = io.imread("/Users/thomasathey/Documents/mimlab/mouselight/BIL/test_synaptomes/stuck.tif")
im_probs = io.imread("/Users/thomasathey/Documents/mimlab/mouselight/BIL/test_synaptomes/stuck_probs.tif")
#im_labels = io.imread("/Users/thomasathey/Documents/mimlab/mouselight/BIL/test_synaptomes/slice_labels.tif")

print("Displaying")
v = napari.Viewer()
v.add_image(im, name="image", contrast_limits=[0, 2**16])
v.add_image(im_probs, name="probs")
# v.add_labels(im_labels)

Displaying


<Image layer 'probs' at 0x158d9ed00>

In [11]:
np.unique(im_probs)

array([0.        , 0.01      , 0.02      , 0.03      , 0.04      ,
       0.05      , 0.06      , 0.07      , 0.08      , 0.09      ,
       0.1       , 0.11      , 0.12      , 0.13      , 0.14      ,
       0.15000001, 0.16      , 0.17      , 0.18000001, 0.19      ,
       0.2       , 0.20999999, 0.22      , 0.23      , 0.23999999,
       0.25      , 0.25999999, 0.27000001, 0.28      , 0.28999999,
       0.30000001, 0.31      , 0.31999999, 0.33000001, 0.34      ,
       0.34999999, 0.36000001, 0.37      , 0.38      , 0.38999999,
       0.40000001, 0.41      , 0.41999999, 0.43000001, 0.44      ,
       0.44999999, 0.46000001, 0.47      , 0.47999999, 0.49000001,
       0.5       , 0.50999999, 0.51999998, 0.52999997, 0.54000002,
       0.55000001, 0.56      , 0.56999999, 0.57999998, 0.58999997,
       0.60000002, 0.61000001, 0.62      , 0.63      , 0.63999999,
       0.64999998, 0.66000003, 0.67000002, 0.68000001, 0.69      ,
       0.69999999, 0.70999998, 0.72000003, 0.73000002, 0.74000

## test parallel zarr write

In [5]:
from joblib import Parallel, delayed
import zarr


def write(path, val):
    z = zarr.open(path, 'w')
    print(z)
    return val

path = "/Users/thomasathey/Documents/mimlab/mouselight/BIL/test.zarr"
z = zarr.zeros((10), chunks=(1), dtype='i4')
zarr.save(path, z)


# results = Parallel(n_jobs=2)(
#         delayed(write)(
#             path, val
#         )
#         for val in range(10)
#     )

In [6]:
z

<zarr.core.Array (10,) int32>